In [1]:
import os
import pickle

import tqdm
import numpy as np
import pandas as pd
import torch
import torchmetrics
import logging
import pytorch_lightning as pl

from copy import deepcopy
from functools import partial

from omegaconf import DictConfig

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from pytorch_lightning.loggers import TensorBoardLogger

from wtte_lib.wtte_data_preprocessing import data_pipeline
from ptls.nn import TrxEncoder, RnnSeqEncoder, Head, L2NormEncoder
from ptls.data_load.utils import collate_feature_dict
from ptls.data_load.datasets import MemoryMapDataset, AugmentationDataset
from ptls.data_load.padded_batch import PaddedBatch
from ptls.data_load.iterable_processing import SeqLenFilter
from ptls.preprocessing import PandasDataPreprocessor
from ptls.frames import PtlsDataModule
from ptls.frames.coles import CoLESModule, ColesDataset
from ptls.frames.coles.losses import SoftmaxLoss
from ptls.frames.coles.metric import BatchRecallTopK
from ptls.frames.coles.split_strategy import SampleSlices
from ptls.frames.inference_module import InferenceModule
from ptls.data_load.augmentations import RandomSlice, DropoutTrx
import warnings
warnings.filterwarnings("ignore")


libgomp: Invalid value for environment variable OMP_NUM_THREADS

libgomp: Invalid value for environment variable OMP_NUM_THREADS


In [2]:
transactions_df = pd.read_csv('../data/transactions.csv')
clients_df = pd.read_csv('../data/clients.csv')
train_df = pd.read_csv('../data/train.csv')
reports_df = pd.read_csv('../data/report_dates.csv')
transactions_df['transaction_dttm'] = pd.to_datetime(transactions_df.transaction_dttm)
transactions_df['transaction_dttm'] = pd.to_datetime(transactions_df['transaction_dttm'], unit='s').astype('int') // 10**9
transactions_df['mcc_code'] += 1 
transactions_df['ones'] = 1

In [3]:
timestep_aggregation_dict = {'ones': 'sum', 'mcc_code': 'max', 'transaction_amt': 'mean', 'currency_rk': 'max'}
train_data = []
transactions_df_ = transactions_df.merge(clients_df[['user_id', 'report']])
for report in reports_df.report.values:
    data_ = transactions_df_[transactions_df_['report']==report].copy().reset_index()
    df_ = data_pipeline(data_, id_col='user_id', infer_seq_endtime=False, abs_time_col='transaction_dttm', column_names=["ones", 'mcc_code', 'transaction_amt', 'currency_rk'], timestep_aggregation_dict=timestep_aggregation_dict)
    train_data.append(df_)

In [4]:
for data in train_data:
    data[0][:,-1,:] = np.array([-1, -1, -1, -1])
data_lst = []
for data in train_data:
    x_ = np.nan_to_num(data[0], 0).copy()
    x_lst = [pd.DataFrame(x_[i]) for i in range(len(x_))]
    for df in x_lst:
        df['target'] = df[2].map(lambda x: 0 if x else None)
        target = df.target.values
        indices = np.where(~np.isnan(target))[0]
        indices[-1]+=1
        idx = 0
        for i, tgt in enumerate(target):
            if np.isnan(tgt):
                target[i] = indices[idx] - i
            else:
                idx+=1
        df['target'] = target
        df.loc[df.index[-1]] = [0, 0, 0, 0, 1]
    for i in range(len(x_lst)):
        x_lst[i]['user_id'] = [data[2][i]]*x_lst[i].shape[0]
    data_lst.append(pd.concat(x_lst, axis=0))

In [5]:
dataset = pd.concat(data_lst, axis=0)
dataset = dataset.rename(columns={0: 'ones', 1: 'mcc_code', 2: 'transaction_amt', 3: 'currency_rk'})
dataset['trx_dt'] = dataset.groupby('user_id').cumcount()+1

In [6]:
from ptls.preprocessing import PandasDataPreprocessor

preprocessor = PandasDataPreprocessor(
    col_id='user_id',
    col_event_time='trx_dt',
    event_time_transformation='none',
    cols_category=['mcc_code', 'currency_rk'],
    cols_numerical=['transaction_amt', 'ones'],
    return_records=True,
)

In [7]:
tr_dataset = preprocessor.fit_transform(dataset)

In [8]:
df = pd.DataFrame(tr_dataset).drop('target', axis=1).merge(train_df[['user_id', 'target', 'time']], on='user_id')
test_ids = pd.read_csv('../data/test_ids.csv')
df_train, df_test = df.loc[~df['user_id'].isin(test_ids['user_id'])], df.loc[df['user_id'].isin(test_ids['user_id'])]

## Supervised Finetune

In [9]:
trx_encoder_params = dict(
    embeddings_noise=0.003,
    numeric_values={'transaction_amt': 'identity',
                    'ones': 'identity',
                   },
    embeddings={
        'currency_rk': {'in': 5, 'out': 4},
        'mcc_code': {'in': 333, 'out': 8},
    },
)

seq_encoder = RnnSeqEncoder(
    trx_encoder=TrxEncoder(**trx_encoder_params),
    hidden_size=800,
    type='gru',
)


In [10]:
class SequenceToTarget(pl.LightningModule):
    def __init__(
        self,
        seq_encoder: torch.nn.Module,
        head: torch.nn.Module=None,
        head_time: torch.nn.Module=None,
        loss: torch.nn.Module=None,
        metric_list: torchmetrics.Metric=None,
        optimizer_partial=None,
        lr_scheduler_partial=None,
        pretrained_lr=None,
        train_update_n_steps=None
    ):
        super().__init__()

        self.save_hyperparameters(ignore=[
            'seq_encoder', 'head', 'head_time', 'loss',
            'metric_list', 'optimizer_partial', 'lr_scheduler_partial'
        ])
        self.seq_encoder = seq_encoder
        self.head = head
        self.head_time = head_time
        self.loss = loss

        if type(metric_list) is dict or type(metric_list) is DictConfig:
            metric_list = [(k, v) for k, v in metric_list.items()]
        else:
            if type(metric_list) is not list:
                metric_list = [metric_list]
            metric_list = [(m.__class__.__name__, m) for m in metric_list]

        self.train_metrics = torch.nn.ModuleDict([(name, deepcopy(mc)) for name, mc in metric_list])
        self.valid_metrics = torch.nn.ModuleDict([(name, deepcopy(mc)) for name, mc in metric_list])

        self.optimizer_partial = optimizer_partial
        self.lr_scheduler_partial = lr_scheduler_partial
        
    def forward(self, x):
        add_features = None
        
        if isinstance(x, tuple):
            x, add_features = x

        x = self.seq_encoder(x)
        
        if self.head is not None:
            y_h = self.head(x)
        else:
            y_h = x
            
        t_h = self.head_time(x)
        
        return y_h, t_h

    def training_step(self, batch, _):
        x, y, t = batch
        y_h, t_h = self(x)

        loss = self.loss(y_h, y)
        self.log('loss/train_loss', loss)
        for name, mf in self.train_metrics.items():
            mf(y_h, y)
            
        loss_t = (t_h - t / 100.0).pow(2).mean()
        self.log('loss/loss_time', loss_t)
        return loss + 0.1 * loss_t

    def training_epoch_end(self, outputs):
        for name, mf in self.train_metrics.items():
            self.log(f'{name}/train', mf.compute(), prog_bar=False)
        for name, mf in self.train_metrics.items():
            mf.reset()

    def validation_step(self, batch, _):
        x, y, t = batch
        y_h, t_h = self(x)
        self.log('loss/valid', self.loss(y_h, y))
        for name, mf in self.valid_metrics.items():
            mf(y_h, y)

    def validation_epoch_end(self, outputs):
        for name, mf in self.valid_metrics.items():
            self.log(f'{name}/valid', mf.compute(), prog_bar=True)
        for name, mf in self.valid_metrics.items():
            mf.reset()

    def configure_optimizers(self):
        if self.hparams.pretrained_lr is not None:
            if self.hparams.pretrained_lr == 'freeze':
                for p in self.seq_encoder.parameters():
                    p.requires_grad = False
                parameters = self.parameters()
            else:
                parameters = [
                    {'params': self.seq_encoder.parameters(), 'lr': self.hparams.pretrained_lr},
                    {'params': self.head.parameters()},  # use predefined lr from `self.optimizer_partial`
                ]
        else:
            parameters = self.parameters()

        optimizer = self.optimizer_partial(parameters)
        scheduler = self.lr_scheduler_partial(optimizer)
        return [optimizer], [scheduler]

In [11]:
class SeqToTargetDataset(torch.utils.data.Dataset):
    def __init__(self,
                 data,
                 target_col_name,
                 target_dtype=None,
                 *args, **kwargs,
                 ):
        super().__init__(*args, **kwargs)

        self.data = data
        
        self.target_col_name = target_col_name
        if type(target_dtype) is str:
            self.target_dtype = getattr(torch, target_dtype)
        else:
            self.target_dtype = target_dtype

    def __len__(self):
        return len(self.data)

    def __getitem__(self, item):
        feature_arrays = self.data[item]
        return feature_arrays

    def __iter__(self):
        for feature_arrays in self.data:
            yield feature_arrays

    def collate_fn(self, padded_batch):
        padded_batch = collate_feature_dict(padded_batch)
        
        target = padded_batch.payload[self.target_col_name]
        time = padded_batch.payload['time']
        del padded_batch.payload[self.target_col_name]
        if self.target_dtype is not None:
            target = target.to(dtype=self.target_dtype)

        return padded_batch, target, time


class SeqToTargetIterableDataset(SeqToTargetDataset, torch.utils.data.IterableDataset):
    pass

In [12]:
def inference(model, dl, device='cuda:0'):
    logits = []
    model.to(device)
    softmax = torch.nn.Softmax(dim=0) 
    for batch in tqdm.tqdm(dl, position=0, leave=True):
        with torch.no_grad():
            x, _, _ = batch
            y_h, t_h = model(x.to(device))
            logits.extend([y_h.cpu()])
        
    logits = softmax(torch.vstack(logits)[:, 1]).cpu()
    
    torch.cuda.empty_cache()

    return logits

In [13]:
skf = StratifiedKFold(n_splits=5)
models = []
predictions_5folds = []

In [14]:
for i, (train_index, test_index) in enumerate(skf.split(df_train.drop(columns=['target']), df_train['target'])):
    train_, test_ = df_train.iloc[train_index], df_train.iloc[test_index]
    
    dataset_train = train_.to_dict(orient='records')
    dataset_test = test_.to_dict(orient='records')
    
    sup_dataset = PtlsDataModule(
        train_data=SeqToTargetDataset(
            AugmentationDataset(
                dataset_train,
                f_augmentations=[
                    DropoutTrx(0.1),
                ],
            ),
            target_col_name='target',
            target_dtype=torch.long,
        ),
        valid_data=SeqToTargetDataset(
            dataset_test,
            target_col_name='target',
            target_dtype=torch.long,
        ),
        train_batch_size=256,
        train_num_workers=8,
        train_drop_last=True,

        valid_batch_size=256,
        valid_num_workers=8,
        valid_drop_last=True
    )
    
    seq_encoder.load_state_dict(torch.load('../models/coles-wtte-model1.pt'))

    sup_module = SequenceToTarget(
        seq_encoder=seq_encoder,
        head=torch.nn.Sequential(
            torch.nn.Linear(seq_encoder.embedding_size, 512),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(512, 2),
            torch.nn.LogSoftmax(dim=1),
        ),
        head_time=torch.nn.Sequential(
            torch.nn.Linear(seq_encoder.embedding_size, 1),
        ),
        loss=torch.nn.NLLLoss(),
        metric_list=torchmetrics.AUROC(num_classes=2),
        optimizer_partial=partial(torch.optim.AdamW, lr=1e-3, weight_decay=0.0),
        lr_scheduler_partial=partial(torch.optim.lr_scheduler.StepLR, step_size=1, gamma=0.2),
    )
    
    
    trainer = pl.Trainer(
        logger=TensorBoardLogger('lightning_logs', name=f'CoLES-WTTE-supervised_{i}'),
        max_epochs=1,
        gpus=1 if torch.cuda.is_available() else 0,
        enable_progress_bar=True,
        gradient_clip_algorithm='norm',
        gradient_clip_val=0.18
    )
    
    trainer.fit(sup_module, sup_dataset)
    
    torch.save(sup_module.state_dict(), f"../models/sup_modules-wtte-kfold.{i}.pt")
    
    predictions_test = test_[["user_id"]].copy()
    
    dataset = SeqToTargetDataset(
        data=dataset_test,
        target_col_name='target',
    )

    dl = torch.utils.data.DataLoader(
        dataset=dataset,
        collate_fn=dataset.collate_fn,
        shuffle=False,
        batch_size=512,
        num_workers=4,
    )
    
    predictions_test["sp"] = inference(sup_module, dl)
    
    predictions_5folds.append(predictions_test)
    
    print(12*"-")
    print("AUROC; 5th fold:", roc_auc_score(test_["target"].values, predictions_test["sp"]))
    print(12*"-")
    
    models.append(deepcopy(sup_module))

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name          | Type          | Params
------------------------------------------------
0 | seq_encoder   | RnnSeqEncoder | 2.0 M 
1 | head          | Sequential    | 411 K 
2 | head_time     | Sequential    | 801   
3 | loss          | NLLLoss       | 0     
4 | train_metrics | ModuleDict    | 0     
5 | valid_metrics | ModuleDict    | 0     
------------------------------------------------
2.4 M     Trainable params
0         Non-trainable params
2.4 M     Total params
9.495     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

100%|██████████| 20/20 [00:01<00:00, 17.61it/s]


...

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

100%|██████████| 20/20 [00:01<00:00, 14.74it/s]


------------
AUROC; 5th fold: 0.5716389017270257
------------


In [15]:
predictions_5folds = pd.concat(predictions_5folds, axis=0)
temp = predictions_5folds.merge(train_df[["user_id", "target"]], on="user_id")
print(12*"-")
print("AUROC; 5th fold:", roc_auc_score(temp["target"].values, temp["sp"].values))
print(12*"-")

------------
AUROC; 5th fold: 0.5986431096538156
------------


In [16]:
dataset_test = df_test.copy()
dataset_test[["target", "time"]] = None
dataset_test = dataset_test.to_dict(orient='records')

dataset = SeqToTargetDataset(
    data=dataset_test,
    target_col_name='target',
)

dl = torch.utils.data.DataLoader(
    dataset=dataset,
    collate_fn=dataset.collate_fn,
    shuffle=False,
    batch_size=512,
    num_workers=4,
)

predictions_test = df_test[["user_id"]].copy()

for i in range(5):
    predictions_test[f"sp_{i}"] = inference(models[i], dl)
predictions_test["sp"] = predictions_test.iloc[:, 1:].mean(axis=1)
predictions_test
print(12*"-")
print("AUROC; 5th fold:", roc_auc_score(df_test["target"].values, predictions_test["sp"].values))
print(12*"-")
train_test_predictions = pd.concat([predictions_5folds, predictions_test[["user_id", "sp"]]], axis=0)
#train_test_predictions.to_csv("sp-preds_train-test.csv", index=False)

100%|██████████| 25/25 [00:01<00:00, 16.09it/s]

------------
AUROC; 5th fold: 0.6531598405182077
------------


---

In [17]:
sample_submit = pd.read_csv("../data/sample_submit_naive.csv")
sbmt_df = pd.DataFrame(tr_dataset).drop('target', axis=1).merge(sample_submit[['user_id']], on='user_id')
sbmt_df[["target", "time"]] = None

In [18]:
sbmt_models = []
for i, (train_index, test_index) in enumerate(skf.split(df.drop(columns=['target']), df["target"])):
    train_, test_ = df.iloc[train_index], df.iloc[test_index]
    
    dataset_train = train_.to_dict(orient='records')
    dataset_test = test_.to_dict(orient='records')
    
    sup_dataset = PtlsDataModule(
        train_data=SeqToTargetDataset(
            AugmentationDataset(
                dataset_train,
                f_augmentations=[
                    DropoutTrx(0.1),
                ],
            ),
            target_col_name='target',
            target_dtype=torch.long,
        ),
        valid_data=SeqToTargetDataset(
            dataset_test,
            target_col_name='target',
            target_dtype=torch.long,
        ),
        train_batch_size=256,
        train_num_workers=8,
        train_drop_last=True,

        valid_batch_size=256,
        valid_num_workers=8,
        valid_drop_last=True
    )
    
    seq_encoder.load_state_dict(torch.load('../models/coles-wtte-model1.pt'))

    sup_module = SequenceToTarget(
        seq_encoder=seq_encoder,
        head=torch.nn.Sequential(
            torch.nn.Linear(seq_encoder.embedding_size, 512),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(512, 2),
            torch.nn.LogSoftmax(dim=1),
        ),
        head_time=torch.nn.Sequential(
            torch.nn.Linear(seq_encoder.embedding_size, 1),
        ),
        loss=torch.nn.NLLLoss(),
        metric_list=torchmetrics.AUROC(num_classes=2),
        optimizer_partial=partial(torch.optim.AdamW, lr=4e-4, weight_decay=0.0),
        lr_scheduler_partial=partial(torch.optim.lr_scheduler.StepLR, step_size=1, gamma=0.2),
    )
    
    trainer = pl.Trainer(
        logger=TensorBoardLogger('lightning_logs', name=f'CoLES-WTTE-supervised-sbmt_{i}'),
        max_epochs=1,
        gpus=1 if torch.cuda.is_available() else 0,
        enable_progress_bar=True,
        gradient_clip_algorithm='norm',
        gradient_clip_val=0.2
    )
    
    trainer.fit(sup_module, sup_dataset)
    
    #torch.save(sup_module.state_dict(), f"model/sup_modules-kfold/sbmt-model-0.{i}.pt")
    
    dataset = SeqToTargetDataset(
        data=dataset_test,
        target_col_name='target',
    )

    dl = torch.utils.data.DataLoader(
        dataset=dataset,
        collate_fn=dataset.collate_fn,
        shuffle=False,
        batch_size=512,
        num_workers=4,
    )
    
    predictions_test = inference(sup_module, dl)
    
    print(12*"-")
    print("AUROC; 5th fold:", roc_auc_score(test_["target"].values, predictions_test))
    print(12*"-")
    
    sbmt_models.append(deepcopy(sup_module))

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name          | Type          | Params
------------------------------------------------
0 | seq_encoder   | RnnSeqEncoder | 2.0 M 
1 | head          | Sequential    | 411 K 
2 | head_time     | Sequential    | 801   
3 | loss          | NLLLoss       | 0     
4 | train_metrics | ModuleDict    | 0     
5 | valid_metrics | ModuleDict    | 0     
------------------------------------------------
2.4 M     Trainable params
0         Non-trainable params
2.4 M     Total params
9.495     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

100%|██████████| 25/25 [00:01<00:00, 13.51it/s]


...

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

100%|██████████| 25/25 [00:01<00:00, 13.24it/s]


------------
AUROC; 5th fold: 0.5692022074710335
------------


In [20]:
dataset_sbmt = sbmt_df.copy()
dataset_sbmt[["target", "time"]] = None
dataset_sbmt = dataset_sbmt.to_dict(orient='records')

dataset = SeqToTargetDataset(
    data=dataset_sbmt,
    target_col_name='target',
)

dl = torch.utils.data.DataLoader(
    dataset=dataset,
    collate_fn=dataset.collate_fn,
    shuffle=False,
    batch_size=512,
    num_workers=4,
)

predictions_sbmt = sbmt_df[["user_id"]].copy()

for i in range(5):
    predictions_sbmt[f"sp_{i}"] = inference(sbmt_models[i], dl)

predictions_sbmt

100%|██████████| 63/63 [00:02<00:00, 22.57it/s]


,user_id,sp_0,sp_1,sp_2,sp_3,sp_4
0,9,0.000038,0.000030,0.000047,0.000057,0.000043
1,61,0.000033,0.000046,0.000033,0.000033,0.000038
2,62,0.000032,0.000048,0.000040,0.000038,0.000041
3,80,0.000014,0.000012,0.000015,0.000014,0.000023
4,88,0.000036,0.000043,0.000050,0.000039,0.000043
...,...,...,...,...,...,...
31995,561362,0.000037,0.000029,0.000036,0.000030,0.000024
31996,561419,0.000029,0.000027,0.000038,0.000051,0.000031
31997,561895,0.000018,0.000024,0.000018,0.000017,0.000031
31998,561908,0.000042,0.000045,0.000034,0.000039,0.000034


In [21]:
predictions_sbmt["sp"] = predictions_sbmt.iloc[:, 1:].mean(axis=1)
predictions_sbmt

,user_id,sp_0,sp_1,sp_2,sp_3,sp_4,sp
0,9,0.000038,0.000030,0.000047,0.000057,0.000043,0.000043
1,61,0.000033,0.000046,0.000033,0.000033,0.000038,0.000037
2,62,0.000032,0.000048,0.000040,0.000038,0.000041,0.000040
3,80,0.000014,0.000012,0.000015,0.000014,0.000023,0.000016
4,88,0.000036,0.000043,0.000050,0.000039,0.000043,0.000042
...,...,...,...,...,...,...,...
31995,561362,0.000037,0.000029,0.000036,0.000030,0.000024,0.000031
31996,561419,0.000029,0.000027,0.000038,0.000051,0.000031,0.000035
31997,561895,0.000018,0.000024,0.000018,0.000017,0.000031,0.000022
31998,561908,0.000042,0.000045,0.000034,0.000039,0.000034,0.000039


In [22]:
final_prediction = pd.concat([predictions_sbmt[["user_id", "sp"]], train_test_predictions])
final_prediction.to_csv("../predicts/sup-wtte-preds.csv", index=False)